In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import coo_matrix



pd.set_option('display.max_colwidth', None)

In [3]:
basepath = '../../datasets/preprocessed_datasets/gabor/'
data = pd.read_pickle(basepath +'user_item_interactions.pkl')
data

,customer_id,product_id,amount
0,1940761,4839681,1
1,1940761,4839682,1
2,1940761,6523930,1
3,1940761,6596395,1
4,1940761,8642222,1
...,...,...,...
644783,11128024,10560958,1
644784,11128039,10451840,1
644785,11128082,10559336,1
644786,11128095,10504300,1


In [15]:
product_data = pd.read_pickle(basepath +'only_products.pkl')
product_data

,product__id,productNumber,productName,productSizeEu,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,...,shoeStyle,productType_y,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor
0,557559,4058394021466,sportliche Ballerinas Glattleder schwarz,40.0,schwarz,02.643.57,6590678,315571,w,PT,...,Ballerinas,Damenschuhe,bis 3 cm,40.0,6.5,11.0,0.0,Gummi,0.0,0.0
30,551622,4054452768212,elegante Pumps Glattleder schwarz,38.0,schwarz,05.160.37,6590677,315573,w,PT,...,Pumps,Damenschuhe,3 cm - 5 cm,38.0,5.0,8.0,0.0,EVA,0.0,0.0
108,547193,4054452768427,Slipper Glattleder schwarz,42.0,schwarz,04.443.27,6590677,315576,w,SK,...,Halbschuhe,Damenschuhe,3 cm - 5 cm,42.0,8.0,14.0,0.0,PU-TPU,0.0,0.0
205,549776,4059701687894,Sneaker low Rauleder blau,39.0,blau,06.968.46,4997827,315567,w,VN,...,Sneaker,Damenschuhe,3 cm - 5 cm,39.0,6.0,10.0,0.0,Gummi-EVA,0.0,0.0
241,550599,4056067216898,Sneaker low Rauleder blau,42.5,blau,06.385.26,6590678,315567,w,VN,...,Sneaker,Damenschuhe,bis 3 cm,42.5,8.5,15.0,0.0,Gummi-EVA,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
648486,10587121,4065171087799,Slipper Rauleder beige,44.0,beige,25.212.12,6590677,315576,w,PT,...,Halbschuhe,Damenschuhe,bis 3 cm,44.0,9.5,17.0,0.0,TPU,0.0,0.0
648487,10366497,4065171994493,Sneaker low Materialmix Leder weiß,43.0,weiß,26.478.51,6590678,315567,w,PT,...,Sneaker,Damenschuhe,3 cm - 5 cm,43.0,9.0,16.0,0.0,TR,0.0,0.0
648489,10227216,4065171789143,PG8000 Sneaker low Materialmix Leder/Textil grün,41.0,grün,8000.14.15,4997827,3692261,m,CN,...,Sneaker,Herrenschuhe,3 cm - 5 cm,41.0,7.5,6.0,0.0,Gummi-EVA,0.0,0.0
648490,10230388,4065171814074,PG0460 Sneaker low Textil blau,44.5,blau,0460.20.02,3692257,3692261,m,PT,...,Sneaker,Herrenschuhe,3 cm - 5 cm,44.5,10.0,11.0,0.0,TR,0.0,0.0


In [4]:
encoder = LabelEncoder()
data['customer_id_encoded'] = encoder.fit_transform(data['customer_id'])
data['product_id_encoded'] = encoder.fit_transform(data['product_id'])

# Create a user-item matrix
user_item_matrix = coo_matrix((data['amount'], (data['customer_id_encoded'], data['product_id_encoded'])))
user_item_matrix = user_item_matrix.tocsr()

# Calculate the cosine similarity between products
product_similarity = cosine_similarity(user_item_matrix.T)


In [28]:
def recommend_products(user_id, n=5):
    user_ratings = user_item_matrix[user_id].toarray().flatten()
    similar_products = product_similarity[user_ratings == 0].sum(axis=0)
    recommended_product_indices = similar_products.argsort()[-n:][::-1]
    recommended_products = encoder.inverse_transform(recommended_product_indices)
    recommended_scores = similar_products[recommended_product_indices]
    return recommended_products, recommended_scores

def get_products_from_customer(cust_id):
    return data[data['customer_id_encoded'] == cust_id]

# Example: Recommend products for a specific user
user_id_to_recommend = 7963  # Change this to the desired user ID
recommended_products, recommended_scores = recommend_products(user_id_to_recommend)


In [29]:
print(f"Recommended products for user {user_id_to_recommend}:")
for i, (product_id, score) in enumerate(zip(recommended_products, recommended_scores)):
    product = product_data[product_data['product__id'] == product_id]
    print(f"{i+1}. Product ID: {product_id}, Score: {score:.4f}, product name: {product.productName}")


Recommended products for user 7963:
1. Product ID: 4085521, Score: 43.7362, product name: 104926    Sneaker high Materialmix Leder braun
Name: productName, dtype: object
2. Product ID: 3711672, Score: 41.9317, product name: 126239    Plateau Pump Leder Reptiloptik schwarz
Name: productName, dtype: object
3. Product ID: 4776107, Score: 41.9317, product name: 274386    Keilpumps Rauleder blau
Name: productName, dtype: object
4. Product ID: 3711325, Score: 41.9317, product name: 162031    Slipper Glattleder schwarz
Name: productName, dtype: object
5. Product ID: 6554512, Score: 41.9317, product name: 370124    Slipper Rauleder blau
Name: productName, dtype: object


In [30]:

customer_products = get_products_from_customer(user_id_to_recommend)
customer_products_data = product_data[product_data['product__id'].isin(customer_products.product_id)]
display(customer_products_data)

,product__id,productNumber,productName,productSizeEu,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,...,shoeStyle,productType_y,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor
253599,5337332,4064032019207,Pantolette Glattleder beige,38.5,beige,62.790.52,6590678,315572,w,PT,...,Sandalen,Damenschuhe,bis 3 cm,38.5,5.5,9.0,0.0,TPU,0.0,0.0


Recommended products for user 1000:
1. Product ID: 4085521, Score: 43.7362, product name: 104926    Sneaker high Materialmix Leder braun
Name: productName, dtype: object
2. Product ID: 3711325, Score: 41.9317, product name: 162031    Slipper Glattleder schwarz
Name: productName, dtype: object
3. Product ID: 3711672, Score: 41.9317, product name: 126239    Plateau Pump Leder Reptiloptik schwarz
Name: productName, dtype: object
4. Product ID: 4776107, Score: 41.9317, product name: 274386    Keilpumps Rauleder blau
Name: productName, dtype: object
5. Product ID: 5507440, Score: 41.9317, product name: 283005    Pantolette Rauleder rot
Name: productName, dtype: object


Recommended products for user 345:
1. Product ID: 4085521, Score: 43.7362, product name: 104926    Sneaker high Materialmix Leder braun
Name: productName, dtype: object
2. Product ID: 6113537, Score: 41.9317, product name: 364202    Plateau Stiefelette Rauleder braun
Name: productName, dtype: object
3. Product ID: 3711672, Score: 41.9317, product name: 126239    Plateau Pump Leder Reptiloptik schwarz
Name: productName, dtype: object
4. Product ID: 3711325, Score: 41.9317, product name: 162031    Slipper Glattleder schwarz
Name: productName, dtype: object
5. Product ID: 5507440, Score: 41.9317, product name: 283005    Pantolette Rauleder rot
Name: productName, dtype: object


Recommended products for user 7963:
1. Product ID: 4085521, Score: 43.7362, product name: 104926    Sneaker high Materialmix Leder braun
Name: productName, dtype: object
2. Product ID: 3711672, Score: 41.9317, product name: 126239    Plateau Pump Leder Reptiloptik schwarz
Name: productName, dtype: object
3. Product ID: 4776107, Score: 41.9317, product name: 274386    Keilpumps Rauleder blau
Name: productName, dtype: object
4. Product ID: 3711325, Score: 41.9317, product name: 162031    Slipper Glattleder schwarz
Name: productName, dtype: object
5. Product ID: 6554512, Score: 41.9317, product name: 370124    Slipper Rauleder blau
Name: productName, dtype: object
